In [ ]:
%matplotlib inline

import os

import mne

## Working in source space in MNE

Localising activity in [source space](https://mne.tools/stable/documentation/glossary.html#term-source-space) is a fundamental part of many EEG and MEG analyses.

Source space localisation generally involves:
1. Computing a subject-specific [forward model](https://mne.tools/stable/documentation/glossary.html#term-forward-solution)
2. Using the forward model to generate an [inverse model](https://mne.tools/stable/documentation/glossary.html#term-inverse-operator) for translating from sensor space to source space

### Part 1 - Creating forward models

Although MNE provides sample forward models, creating these based on MRI recordings of each individual subject offers greater accuracy when localising sources.

Thankfully, MNE offers a number of tools for creating subject-specific forward models.

Computing forward models requires:
- [Coregistration information](https://mne.tools/stable/documentation/glossary.html#term-trans) (stored as a `-trans.fif` file) for aligning head and sensor positions
- The boundary element model ([BEM](https://mne.tools/stable/documentation/glossary.html#term-BEM)) surfaces which influence how source activity propagates to the sensors
- A source space of locations in the brain at which to estimate source activity

We start by establishing the sample data paths.

In [ ]:
# Path to the sample data
sample_data_path = mne.datasets.sample.data_path()
sample_dir = sample_data_path / "MEG" / "sample"

# Path to the FreeSurfer reconstructions
subjects_dir = os.path.join(sample_data_path, "subjects")
subject = "sample"

#### Transformation information from coregistration

Coregistration is the process of aligning head and sensor locations in a common coordinate system.

This can be performed using the [`mne.gui.coregistration()`](https://mne.tools/stable/generated/mne.gui.coregistration.html) function.

Run the cell below to open the coregistration GUI.

In [ ]:
mne.gui.coregistration(subject=subject, subjects_dir=subjects_dir);

Below you can view a transformation file from a previous coregisration for the sample data.

What do the coloured dots represent?

What do the blue panels represent?

In [ ]:
# Load the transformation file obtained by coregistration
transformation = os.path.join(sample_dir, "sample_audvis_raw-trans.fif")

# Load the data's information
raw = mne.io.read_raw_fif(os.path.join(sample_dir, "sample_audvis_raw.fif"))

# Visualise coregistration of head and sensors
mne.viz.plot_alignment(
    info=raw.info,
    trans=transformation,
    subject=subject,
    subjects_dir=subjects_dir,
    surfaces="head-dense",
    meg=["helmet", "sensors"],
    dig=True,
);

#### Computing the BEM Surfaces

BEM surfaces are triangulations of the interfaces between different tissues which affect the propagation of signals (e.g. inner skull surface, outer skull surface, scalp surface).

Computing BEM surfaces makes use of [FreeSurfer](https://surfer.nmr.mgh.harvard.edu/), and can be performed using the [`mne.bem.make_flash_bem()`](https://mne.tools/stable/generated/mne.bem.make_flash_bem.html) or [`mne.bem.make_watershed_bem()`](https://mne.tools/stable/generated/mne.bem.make_watershed_bem.html) functions.

As this takes several minutes to compute per subject, we can take advantage of the pre-existing surfaces.

We plot these surfaces using the [`mne.viz.plot_bem()`](https://mne.tools/stable/generated/mne.viz.plot_bem.html) function.

What do the coloured lines represent?

In [ ]:
mne.viz.plot_bem(
    subject=subject,
    subjects_dir=subjects_dir,
    orientation="coronal",
    slices=[50, 100, 150, 200],
    brain_surfaces="white",
);

#### Defining the source space

The source space determines the position and orientation of candidate source locations.

Source spaces can be:
- Surface-based - source candidates are confined to a surface, e.g. the cortical surface ([`mne.setup_source_space()`](https://mne.tools/stable/generated/mne.setup_source_space.html))
- Volumetric/discrete - source candidates are arbitrary points within an area, e.g. within the inner skull ([`mne.setup_volume_source_space()`](https://mne.tools/stable/generated/mne.setup_volume_source_space.html))

Below, we create source space candidates on the cortical surface and visualise them in 3D.

In [ ]:
# Create source space candidates
surface_sources = mne.setup_source_space(
    subject, spacing="oct6", subjects_dir=subjects_dir, add_dist="patch"
)

# Visualise the candidates in 3D
mne.viz.plot_alignment(
    subject=subject,
    subjects_dir=subjects_dir,
    surfaces="white",
    coord_frame="mri",
    src=surface_sources,
);

When creating volumetric source space candidates, these can be bound to a limited area.

For example, we can create candidate sources only within the brain using the BEM surface of the inner skull, which we can visualise alongside the BEM surfaces.

In [ ]:
# Load the BEM surface for the inner skull
inner_skull_surface = os.path.join(subjects_dir, subject, "bem", "inner_skull.surf")

# Create the volumetric source candidates
volume_sources = mne.setup_volume_source_space(
    subject=subject, surface=inner_skull_surface, subjects_dir=subjects_dir, add_interpolator=False
)

# Visualise BEM surfaces and source space candidates (purple dots)
mne.viz.plot_bem(
    subject=subject,
    subjects_dir=subjects_dir,
    orientation="coronal",
    slices=[50, 100, 150, 200],
    brain_surfaces="white",
    src=volume_sources,
);

What would happen if we did not specify a surface to bound the candidates to?

#### Computing the forward model

With the coregistration, BEM surfaces, and source space sorted, we can at last compute the forward model.

We first load the BEM surfaces to create a BEM model using the [`mne.make_bem_model()`](https://mne.tools/stable/generated/mne.make_bem_model.html) function.

We then create a BEM solution from this using the [`mne.make_bem_solution()`](https://mne.tools/stable/generated/mne.make_bem_solution.html) function.

In [ ]:
# Use only a single-layer BEM (just the inner skull) for speed
conductivity = 0.3

# Create the BEM model
bem_model = mne.make_bem_model(
    subject=subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir
)

# Use the BEM model to create the BEM solution
bem_solution = mne.make_bem_solution(bem_model)

Finally, we can pass the BEM solution along with our source space candidates and coregistration information to the [`mne.make_forward_solution()`](https://mne.tools/stable/generated/mne.make_forward_solution.html) function to create the forward model.

Here, we only compute the model for the MEG sensors, using the surface-based (i.e. cortical) source candidates.

In [ ]:
# Create the forward model
forward = mne.make_forward_solution(
    info=raw.info,  # data information
    trans=transformation,  # coregistration information
    src=surface_sources,  # surface-based source candidates
    bem=bem_solution,  # BEM solution
    meg=True,
    eeg=False,
)

This returns an [`mne.Forward`](https://mne.tools/stable/generated/mne.Forward.html) object, which contains the forward model.

In [ ]:
forward

The leadfield matrix representing the transformation between source and sensor spaces can be extracted from the `Forward` object as below.

In [ ]:
leadfield = forward["sol"]["data"]
print(f"Leadfield matrix shape: {leadfield.shape} (sensors x dipoles)")

Notice how the number of dipoles (24,579) is 3 times greater than the number of sources (8,193).

This reflects the fact that we have not specified the orientations of the sources, so the leadfield matrix is computed for each orientation in 3D space.

Fixing the orientation of the sources (e.g. to a cortical orientation) can be performed using the [`mne.convert_forward_solution()`](https://mne.tools/stable/generated/mne.convert_forward_solution.html) function.

### Part 2 - Inverse modelling for source localisation

Now that we have a forward model, we can use this to reconstruct source activity from sensor space data.

Various approaches for source reconstruction are offered in MNE:
- Minimum norm estimation ([MNE](https://mne.tools/stable/documentation/glossary.html#term-MNE))
- Dynamic statistical parametric mapping ([dSPM](https://mne.tools/stable/documentation/glossary.html#term-dSPM))
- Standardised low-resolution electromagnetic tomography ([sLORETA](https://mne.tools/stable/documentation/glossary.html#term-sLORETA))
- Exact low-resolution electromagnetic tomography ([eLORETA](https://mne.tools/stable/documentation/glossary.html#term-eLORETA))
- Linearly constrained minimum variance ([LCMV](https://mne.tools/stable/documentation/glossary.html#term-LCMV)) [beamformer](https://mne.tools/stable/documentation/glossary.html#term-beamformer)
- Dynamic imaging of coherent sources ([DICS](https://mne.tools/stable/documentation/glossary.html#term-DICS)) beamformer

<br>

Creating an inverse model requires:
- A forward model
- A covariance matrix

<br>

Below we will explore the process for creating an inverse model and applying it with the dSPM approach.

We start by loading MNE's sample data and creating epochs around the left auditory stimuli.

In [ ]:
# Load the sample data
raw = mne.io.read_raw_fif(fname=os.path.join(sample_dir, "sample_audvis_raw.fif"))
raw.pick(picks=["eeg", "stim"], exclude="bads")
raw.set_eeg_reference(ref_channels="average", projection=True)

# Create the events array
events = mne.find_events(raw=raw, stim_channel="STI 014")

# Create the epochs
epochs = mne.Epochs(
    raw=raw,
    events=events,
    event_id={"auditory/left": 1},  # isolate the left auditory stimuli
    tmin=-0.2,  # start each epoch 200 ms before the stimulus
    tmax=0.5,  # end each epoch 500 ms after the stimulus
    baseline=(None, 0),  # baseline epochs in the window [-200, 0] ms
)

Here, we average over the epochs to create ERPs in sensor space.

This is the activity we will reconstruct in source space.

In [ ]:
# Create ERPs from the epochs
evoked = epochs.average()
evoked.plot()
evoked.plot_topomap(times=[-0.1, 0.0, 0.1, 0.2]);

As we are working with MNE's sample data, we can load the pre-computed forward model using [`mne.read_forward_solution()`](https://mne.tools/stable/generated/mne.read_forward_solution.html).

In [ ]:
# Load pre-computed EEG forward solution
forward = mne.read_forward_solution(os.path.join(sample_dir, "sample_audvis-eeg-oct-6-fwd.fif"))
forward

Next, we need to compute the covariance matrix - specifically the [covariance matrix of the noise](https://mne.tools/stable/documentation/glossary.html#term-noise-covariance) - using the [`mne.compute_covariance()`](https://mne.tools/stable/generated/mne.compute_covariance.html) function.

Here, we define noise to be the baseline period preceding stimulus presentation, and as such specify that covariance should only be computed for the window [-0.2, 0] seconds.

What does the covariance matrix represent, and what role does it play in the inverse model?

In [ ]:
# Compute and plot noise covariance
noise_covariance = mne.compute_covariance(epochs=epochs, tmin=None, tmax=0, method="empirical")
mne.viz.plot_cov(noise_covariance, raw.info);

With the forward model and covariance matrix, we can create an inverse model using the [`mne.minimum_norm.make_inverse_operator()`](https://mne.tools/stable/generated/mne.minimum_norm.make_inverse_operator.html) function.

In [ ]:
# Create inverse model
inverse = mne.minimum_norm.make_inverse_operator(
    info=evoked.info, forward=forward, noise_cov=noise_covariance
)
inverse

The inverse model can be applied to:
- `Evoked` objects - [`mne.minimum_norm.apply_inverse()`](https://mne.tools/stable/generated/mne.minimum_norm.apply_inverse.html)
- `Raw` objects - [`mne.minimum_norm.apply_inverse_raw()`](https://mne.tools/stable/generated/mne.minimum_norm.apply_inverse_raw.html)
- `Epochs` objects - [`mne.minimum_norm.apply_inverse_epochs()`](https://mne.tools/stable/generated/mne.minimum_norm.apply_inverse_epochs.html)
- `EpochsTFR` objects - [`mne.minimum_norm.apply_inverse_tfr_epochs()`](https://mne.tools/stable/generated/mne.minimum_norm.apply_inverse_tfr_epochs.html)
- `Covariance` objects - [`mne.minimum_norm.apply_inverse_cov()`](https://mne.tools/stable/generated/mne.minimum_norm.apply_inverse_cov.html)

Here, we apply the inverse model to the ERP data using the dSPM method, which returns an [`mne.SourceEstimate`](https://mne.tools/stable/generated/mne.SourceEstimate.html) object.

In [ ]:
# Extract source activity of the ERP
source_activity = mne.minimum_norm.apply_inverse(
    evoked=evoked, inverse_operator=inverse, method="dSPM"
)
source_activity

We can now visualise the ERPs in source space using the [`plot()`](https://mne.tools/stable/generated/mne.SourceEstimate.html#mne.SourceEstimate.plot) method.

Play around with the options in the visualisation GUI to see how the [source activity](https://mne.tools/stable/documentation/glossary.html#term-source-estimate) changes over time in response to the stimuli.

Does the localisation of this source activity make sense given the simulus being presented?

In [ ]:
# Visualise source activity
source_activity.plot(
    subject=subject, hemi="rh", subjects_dir=subjects_dir, initial_time=0.1, backend="pyvistaqt"
);

## Conclusion

Reconstructing activity in source space is a fundamental part of many neuroscience projects involving EEG and MEG data.

As you can see, there are a wide range of tools available in MNE for translating activity from sensor space to source space, which we have only briefly covered here.

For more in-depth discussions of particular approaches, see the MNE tutorials linked below.

## Additional resources

MNE forward modelling tutorials: https://mne.tools/stable/auto_tutorials/forward/index.html

MNE inverse modelling tutorials: https://mne.tools/stable/auto_tutorials/inverse/index.html

MNE forward modelling module: https://mne.tools/stable/api/forward.html

MNE inverse modelling module: https://mne.tools/stable/api/inverse.html